In [57]:
from dataclasses import dataclass
from typing import Dict, List, Tuple, Union
from random import choices, randint, uniform

from midiutil import MIDIFile

In [60]:
NOTE_LENGTH = [1 / 32, 1 / 16, 1 / 8, 1 / 4]  # w.r.t. bar
NOTE_PROB = [0.0, 0.8, 0.2, 0.0]

SILENCE_PROB = 0.25

MELODY_PATTERNS = ["AABA", "ABAB", "ABAC"]

In [28]:
temp = {}

for _ in range(2000):
    choice = choices(NOTE_LENGTH, NOTE_PROB)[0]
    if choice in temp:
        temp[choice] += 1
    else:
        temp[choice] = 1
        
for c in temp:
    temp[c] = temp[c] / 2000.0

In [68]:
@dataclass
class MidiData:
    note: int
    start_time: float
    duration: float

major_scale = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number
minor_scale = [60, 62, 63, 65, 67, 68, 70, 72] # MIDI note number
BEAT_PER_BAR = 4
VOLUME = 100

def generate_midi(scale: str='C', major: bool=True, bpm: int=126, bars: int=4):
    melody_pattern = MELODY_PATTERNS[randint(0, len(MELODY_PATTERNS) - 1)]
    
    pattern_memo: Dict[str, List[MidiData]] = {}
    for pattern in melody_pattern:
        if pattern in pattern_memo:
            continue
        time = 0
        pattern_memo[pattern] = []
        while time < BEAT_PER_BAR:
            is_silence = uniform(0, 1) <= SILENCE_PROB
            if is_silence:
                duration = choices(NOTE_LENGTH, NOTE_PROB)[0] * BEAT_PER_BAR
                time += duration
                continue
            note_id = randint(0, len(major_scale) - 1)
            note = major_scale[note_id] if major else minor_scale[note_id]
            duration = choices(NOTE_LENGTH, NOTE_PROB)[0] * BEAT_PER_BAR
            if time + duration > BEAT_PER_BAR:
                duration = BEAT_PER_BAR - time
            pattern_memo[pattern].append(MidiData(note=note, start_time=time, duration=duration))
            time += duration
    
    midi = MIDIFile(1)
    midi.addTempo(0, 0, bpm)
    bar_num = 0
    for pattern in melody_pattern:
        for midi_data in pattern_memo[pattern]:
            time = midi_data.start_time + bar_num * BEAT_PER_BAR
            midi.addNote(0, 0, midi_data.note, time, midi_data.duration, VOLUME)
        bar_num += 1
    with open("minor-scale.mid", "wb") as output_file:
        midi.writeFile(output_file)

generate_midi(major=False)

0 60 0.25
0.5 70 0.5
2.0 65 0.25
2.25 68 0.25
2.5 62 0.25
2.75 68 0.25
3.25 72 0.25
4 60 0.25
4.5 70 0.5
6.0 65 0.25
6.25 68 0.25
6.5 62 0.25
6.75 68 0.25
7.25 72 0.25
8 60 0.5
8.5 60 0.25
8.75 60 0.5
9.25 70 0.5
9.75 72 0.25
10.0 72 0.25
11.0 72 0.25
11.25 62 0.25
11.5 62 0.25
11.75 63 0.25
12 60 0.25
12.5 70 0.5
14.0 65 0.25
14.25 68 0.25
14.5 62 0.25
14.75 68 0.25
15.25 72 0.25


In [31]:
degrees  = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number

midi = MIDIFile(1)
midi.addTempo(0, 0, 126)
duration = 1 / 4
volume = 100
time = 0

for pitch in degrees:
    midi.addNote(0, 0, pitch, time, duration, volume)
    time += duration
    
with open("major-scale.mid", "wb") as output_file:
    midi.writeFile(output_file)